## 추천시스템이란 게 뭔가요?

In [2]:
# 콘텐츠 추천 알고리즘의 진화 http://www.kocca.kr/insight/vol05/vol05_04.pdf
# 추천 시스템은 사용자가 선호할 만한 아이템을 추측함으로써 여러 가지 항목 중 사용자에게 적합한 특정 항목을 
# 선택(information filtering)하여 제공하는 시스템을 일컫는다. 
# 여기서 ‘필터링’이란 여러 가지 항목 중 적당한 항목을 선택하는 기술을 말하는 IT 용어이다. 
# (1) 협업 필터링(Collaborative filtering)
#   : 협업 필터링이란 대규모의 기존 사용자 행동 정보(사용자의 행동 기록을 이용)를 분석하여, 
#     해당 사용자와 비슷한 성향의 사용자들이 기존에 좋아했던 항목을 추천하는 기술이다. 
#     가장 일반적인 예는 온라인 쇼핑 사이트에서 흔히 볼 수 있는 ‘이 상품을 구매한 사용자가 구매한 상품들’ 서비스이다. 
#     예를 들어 ‘라면’을 구입한 사용자가 ‘생수’를 구입한 경우가 많으면 ‘라면’을 구입하는 구매자에게 ‘생수’를 추천하는 경우이다. 
#   : [단점] 콜드 스타트(Cold Start), 사용자수가 많으면 계산량이 많아짐. 롱테일(Long tail) 문제 - 비대칭적 쏠림 현상으로 정보 부족
# (2) 콘텐츠 기반 필터링 (Content-based filtering)
#   : 콘텐츠 기반 필터링은 항목 자체를 분석하여 추천을 구현한다. 
#     예를 들어 음악을 추천하기 위해 음악 자체를 분석하여 유사한 음악을 추천하는 방식이다.
#     콘텐츠 기반 필터링을 위해서는 항목을 분석한 프로파일(item profile)과 
#     사용자의 선호도를 추출한 프로파일(user profile)을 추출하여 이의 유사성을 계산한다. 
#   : [단점] 협업 필터링에서 발생하는 콜드 스타트 문제를 해결했지만 다양한 형식의 항목을 추천하기 어려운 단점이 있다. 
#     예를들어 음악과 사진, 비디오를 동시에 추천해야 하는 경우, 각각의 항목에서 얻을 수 있는 정보가 다르기 때문에 
#     프로파일을 구성하기 매우 어려워진다. 
# (3) 모델 기반 협력 필터링(Model-based Collaborative Filtering algorithm)
#   : 기존 항목 간 유사성을 단순하게 비교하는 것에서 벗어나 자료 안에 내재한 패턴을 이용하는 기법
#     1) 연관되는 자료의 크기를 동적으로 변화시키는 방법 (해리포터2 <- 해리포터1 시청여부)
#     2) 잠재(latent) 모델 기반 - 사용자가 특정 항목을 선호하는 이유를 알고리즘적으로 알아내는 기법 (주연배우, 드라마OST, 장르 등)

## (1) 데이터 탐색하기와 전처리
### 데이터 준비

In [3]:
import pandas as pd
import os

fname = os.getenv('HOME') + '/aiffel/recommendata_iu/data/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv'
col_names = ['user_id', 'artist_MBID', 'artist', 'play']   # 임의로 지정한 컬럼명
data = pd.read_csv(fname, sep='\t', names= col_names)      # sep='\t'로 주어야 tsv를 열 수 있습니다.  
data.head(10)

,user_id,artist_MBID,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,8bfac288-ccc5-448d-9573-c33ea2aa5c30,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,6531c8b1-76ea-4141-b270-eb1ac5b41375,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,21f3573f-10cf-44b3-aeaa-26cccd8448b5,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,c5db90c4-580d-4f33-b364-fbaa5a3a58b5,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,0639533a-0402-40ba-b6e0-18b067198b73,lunachicks,403


In [4]:
# 사용하는 컬럼만 남겨줍니다.
using_cols = ['user_id', 'artist', 'play'] #artist MBID 제거
data = data[using_cols]
data.head(10)

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


In [5]:
# 검색을 쉽게하기 위해 아티스트 문자열을 소문자로 바꿔 줍시다.
data['artist'] = data['artist'].str.lower() # 검색을 쉽게하기 위해 아티스트 문자열을 소문자로 바꿔줍시다.
data.head(10)

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


In [6]:
# 첫 번째 유저가 어떤 아티스트의 노래를 듣는지 확인해 봅시다.
condition = data['user_id']== data.loc[0, 'user_id']
data.loc[condition]

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


In [7]:
# 처음 들어보는 아티스트가 많습니다. 추천시스템에서는 적용하는 분야에 대한 지식이 특히 더 필요한데, 모르면 검증을 못 하기 때문입니다.

## 데이터 탐색

In [8]:
# 추천 모델을 만들기 전에 데이터의 기본적인 정보를 보고 갑시다. 아래 항목들을 확인해 보고 싶습니다.
# 유저수, 아티스트수, 인기 많은 아티스트
# 유저들이 몇 명의 아티스트를 듣고 있는지에 대한 통계
# 유저 play 횟수 중앙값에 대한 통계

In [9]:
# pandas.DataFrame.nunique()은 특정 컬럼에 포함된 유니크한 데이터의 개수를 알아보는데 유용합니다.
data['user_id'].nunique() # 유저 수

358868

In [10]:
# 아티스트 수
data['artist'].nunique()

291346

In [11]:
# 인기 많은 아티스트
artist_count = data.groupby('artist')['user_id'].count()
artist_count.sort_values(ascending=False).head(50)

artist
radiohead                77254
the beatles              76245
coldplay                 66658
red hot chili peppers    48924
muse                     46954
metallica                45233
pink floyd               44443
the killers              41229
linkin park              39773
nirvana                  39479
system of a down         37267
queen                    34174
u2                       33206
daft punk                33001
the cure                 32624
led zeppelin             32295
placebo                  32072
depeche mode             31916
david bowie              31862
bob dylan                31799
death cab for cutie      31482
arctic monkeys           30348
foo fighters             30144
air                      29795
the rolling stones       29754
nine inch nails          28946
sigur rós                28901
green day                28732
massive attack           28691
moby                     28232
amy winehouse            28210
portishead               28072
r

In [12]:
# 유저별 몇 명의 아티스트를 듣고 있는지에 대한 통계
user_count = data.groupby('user_id')['artist'].count()
user_count.describe()

count    358868.000000
mean         48.863234
std           8.524272
min           1.000000
25%          46.000000
50%          49.000000
75%          51.000000
max         166.000000
Name: artist, dtype: float64

In [13]:
# 유저별 play횟수 중앙값에 대한 통계
user_median = data.groupby('user_id')['play'].median()
user_median.describe()

count    358868.000000
mean        142.187676
std         213.089902
min           1.000000
25%          32.000000
50%          83.000000
75%         180.000000
max       50142.000000
Name: play, dtype: float64

## 모델 검증을 위한 사용자 초기 정보 세팅

In [14]:
# 우리는 본인의 음악 취향과 가장 유사한 아티스트를 추천받고 싶습니다. 유튜브에선 이를 위해서 어떤 과정을 거쳤나요? 
# 네, 사용자의 취향과 유사한 아티스트 정보를 5개 이상 입력받는 과정을 거쳤습니다.
# 우리도 동일한 과정이 필요합니다. 우리가 만든 추천시스템의 추후 검증 과정을 위해, 
# '나는 A를 좋아한다.'라는 정보를 위 데이터셋에 5개 이상 추가해 주어야 합니다. 
# 저는 아래와 같이 제가 좋아하는 유명한 아티스트들을 기존의 데이터에 추가해서 넣겠습니다.

In [15]:
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['black eyed peas' , 'maroon5' ,'jason mraz' ,'coldplay' ,'beyoncé']

# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['zimin']*5, 'artist': my_favorite, 'play':[30]*5})

if not data.isin({'user_id':['zimin']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,artist,play
17535650,"sep 20, 2008",turbostaat,12
17535651,"sep 20, 2008",cuba missouri,11
17535652,"sep 20, 2008",little man tate,11
17535653,"sep 20, 2008",sigur rós,10
17535654,"sep 20, 2008",the smiths,10
0,zimin,black eyed peas,30
1,zimin,maroon5,30
2,zimin,jason mraz,30
3,zimin,coldplay,30
4,zimin,beyoncé,30


## 내 데이터로 테스트해보기

In [17]:
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['radiohead', 'the beatles', 'coldplay', 'the prodigy', 'madonna']

# 'yesica'이라는 user_id가 위 아티스트의 노래를 50회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['yesica']*5, 'artist': my_favorite, 'play':[50]*5})

if not data.isin({'user_id':['yesica']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,artist,play
0,zimin,black eyed peas,30
1,zimin,maroon5,30
2,zimin,jason mraz,30
3,zimin,coldplay,30
4,zimin,beyoncé,30
0,yesica,radiohead,50
1,yesica,the beatles,50
2,yesica,coldplay,50
3,yesica,the prodigy,50
4,yesica,madonna,50


## 모델에 활용하기 위한 전처리 (실습)

In [18]:
# 사람이 태어나면 주민등록번호가, 학교에 가면 출석번호가 있듯이 데이터의 관리를 쉽게 하기 위해 번호를 붙여주고 싶습니다. 
# 우리가 다루는 데이터에서는 user와 artist 각각에 번호를 붙이고 싶습니다. 
# 보통 이런 작업을 indexing이라고 합니다. 
# 추천시스템, 자연어처리에서 자주 하는 작업들이라 직접 해보시면 좋겠습니다.

In [19]:
# pandas.DataFrame.unique()은 특정 컬럼에 포함된 유니크한 데이터만 모아 줍니다. indexing 작업을 위해 매우 유용합니다.
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = data['user_id'].unique()
artist_unique = data['artist'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
artist_to_idx = {v:k for k,v in enumerate(artist_unique)}

In [22]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['zimin'])    # 358869명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 합니다. 
print(artist_to_idx['black eyed peas'])

358868
376


In [20]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['yesica'])    # 358870명의 유저 중 마지막으로 추가된 유저이니 358869이 나와야 합니다. 
print(artist_to_idx['the beatles'])

358869
355


In [21]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_artist_data = data['artist'].map(artist_to_idx.get).dropna()
if len(temp_artist_data) == len(data):
    print('artist column indexing OK!!')
    data['artist'] = temp_artist_data
else:
    print('artist column indexing Fail!!')

data

user_id column indexing OK!!
artist column indexing OK!!


,user_id,artist,play
0,0,0,2137
1,0,1,1099
2,0,2,897
3,0,3,717
4,0,4,706
...,...,...,...
0,358869,217,50
1,358869,247,50
2,358869,62,50
3,358869,266,50


## 사용자의 명시적/암묵적 평가

In [22]:
# 우리의 추천시스템은 사용자들이 아이템을 얼마나 선호하는지를 모델링하기를 원합니다. 
# 그러자면 사용자의 아이템 선호도를 말해 주는 유저 행동 데이터셋이 필요합니다. 
# 만약 우리 데이터가 좋아요나 별점처럼 선호도를 명시적(explicit)으로 나타내는 것이라면 참 좋겠지만, 
# 지금 우리가 가지고 있는 데이터는 사용자가 아티스트의 곡을 몇번 플레이했나 하는 것 뿐입니다. 
# 이렇게 서비스를 사용하면서 자연스럽게 발생하는 암묵적(implicit)인 피드백도 사용자의 아이템에 대한 평가를 알 수 있는 단서가 될 수 있습니다.

In [23]:
# 우리의 데이터셋에서 1회만 플레이한 데이터의 비율은 어떻게 될까요?
# 1회만 play한 데이터의 비율을 보는 코드
only_one = data[data['play']<2]
one, all_data = len(only_one), len(data)
print(f'{one},{all_data}')
print(f'Ratio of only_one over all data is {one/all_data:.2%}')  # f-format에 대한 설명은 https://bit.ly/2DTLqYU

147740,17535665
Ratio of only_one over all data is 0.84%


In [24]:
# 앞으로 만들어갈 모델에서는 암묵적 데이터의 해석을 위해 다음과 같은 규칙을 적용할 것입니다.
# 1. 한 번이라도 들었으면 선호한다고 판단한다.
# 2. 많이 재생한 아티스트에 대해 가중치를 주어서 더 확실히 좋아한다고 판단한다.

## Matrix Factorization(MF)

In [25]:
# 데이터가 준비되었으니 이제 모델을 만들 차례입니다!
# 추천 시스템의 다양한 모델 중 Matrix Factorization(MF, 행렬분해) 모델을 사용하겠습니다. 
# MF 모델은 2006년 Netflix에서 백만달러의 상금을 걸고 개최한 자사 추천시스템의 성능을 10% 이상 향상시키는 챌린지를 계기로 알려지게 되었습니다. 
# 기본 아이디어는 아래 그림과 같습니다. (m,n) 사이즈의 행렬 R을 (m,k) 사이즈의 행렬 P와 (k,n) 사이즈의 행렬 Q로 분해한다면 
# R이란 그저 P와 Q의 행렬곱으로 표현 가능할 수 있다는 간단한 아이디어입니다. 
# 대체로 k는 m이나 n보다 훨씬 작은 값이기 때문에 계산량 측면으로도 훨씬 유리해집니다. 
# 하지만 아이디어의 단순함에도 불구하고 MF 모델은 성능이 준수하고 Scalability가 좋아서 많이 사용되는 모델입니다.
# 첫번째 사용자의 특성(Feature) 벡터가 됩니다.
# 두번째 영화의 특성(Feature) 벡터가 됩니다. 

## CSR(Compressed Sparse Row) Matrix

In [26]:
# 거대한 행렬을 메모리에 올려 놓고 작업한다는 것은 불가능할 것입니다. 
# 이런 경우의 좋은 대안이 되는 것이 CSR(Compressed Sparse Row) Matrix입니다. 
# 다음 스텝에서 수행할 모델 학습의 input으로 사용할 데이터 타입을 CSR Matrix로 할 것입니다.
# CSR Matrix는 Sparse한 matrix에서 0이 아닌 유효한 데이터로 채워지는 데이터의 값과 좌표 정보로만으로 구성하여 
# 메모리 사용량을 최소화하면서도 Sparse한 matrix와 동일한 행렬을 표현할 수 있도록 하는 데이터 구조입니다. 

In [27]:
# csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])  
# where data, row_ind and col_ind satisfy the relationship a[row_ind[k], col_ind[k]] = data[k]., M,N은 matrix의 shape

# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_artist = data['artist'].nunique()

csr_data = csr_matrix((data.play, (data.user_id, data.artist)), shape= (num_user, num_artist))
csr_data

<358870x291347 sparse matrix of type '<class 'numpy.longlong'>'
	with 17535583 stored elements in Compressed Sparse Row format>

## MF 모델 학습하기

In [28]:
# 이전 스텝에서 설명한 Matrix Factorization 모델을 implicit 패키지를 사용하여 학습해 봅시다.
# implicit 패키지는 이전 스텝에서 설명한 암묵적(implicit) dataset을 사용하는 다양한 모델을 굉장히 빠르게 학습할 수 있는 패키지입니다.
# 이 패키지에 구현된 als(AlternatingLeastSquares) 모델을 사용하겠습니다. 
# Matrix Factorization에서 쪼개진 두 Feature Matrix를 한꺼번에 훈련하는 것은 잘 수렴하지 않기 때문에, 
# 한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행하는 AlternatingLeastSquares 방식이 효과적인 것으로 알려져 있습니다.

In [29]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [30]:
# AlternatingLeastSquares 클래스의 __init__ 파라미터를 살펴보겠습니다. 
# 1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지 
# 2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지 
# 3. use_gpu : GPU를 사용할 것인지 
# 4. iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지
# 1,4를 늘릴수록 학습데이터를 잘 학습하게 되지만 과적합의 우려가 있으니 좋은 값을 찾아야 합니다.

In [31]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [32]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<291347x358870 sparse matrix of type '<class 'numpy.longlong'>'
	with 17535583 stored elements in Compressed Sparse Column format>

In [33]:
# 모델 훈련
als_model.fit(csr_data_transpose)

In [ ]:
# 모델 학습이 끝났습니다!
# 모델이 1)저의 벡터와 black eyed peas의 벡터를 어떻게 만들고 있는지와 
# 2)두 벡터를 곱하면 어떤 값이 나오는지 살펴보겠습니다.

In [57]:
zimin, black_eyed_peas = user_to_idx['zimin'], artist_to_idx['black eyed peas']
zimin_vector, black_eyed_peas_vector = als_model.user_factors[zimin], als_model.item_factors[black_eyed_peas]

In [58]:
yesica, the_beatles = user_to_idx['yesica'], artist_to_idx['the beatles']
yesica_vector, the_beatles_vector = als_model.user_factors[yesica], als_model.item_factors[the_beatles]

In [59]:
zimin_vector

array([ 1.21675476e-01,  4.19114769e-01, -5.49941659e-01,  1.05485415e+00,
       -7.03012586e-01, -2.02309942e+00,  2.72546768e-01, -7.37980783e-01,
        3.88130486e-01, -1.26163229e-01,  7.80983567e-01, -2.64218926e-01,
       -6.26418591e-02, -4.34423178e-01, -2.80600250e-01, -1.14281380e+00,
       -5.04144967e-01,  1.01816446e-01, -2.68255591e-01,  6.06463969e-01,
        4.06078815e-01,  9.70983863e-01, -4.41041738e-01,  7.82994106e-02,
        3.81856740e-01,  9.30189312e-01, -6.95997953e-01, -3.78995001e-01,
       -1.50947556e-01,  2.71626532e-01, -7.19416440e-01, -4.07430530e-01,
       -6.86011859e-04,  7.27658749e-01,  2.66089767e-01, -3.76610428e-01,
        6.84908092e-01,  1.18859959e+00, -1.89952478e-01,  3.01615089e-01,
        5.51060021e-01, -1.25782239e+00,  1.63262701e+00, -9.76854935e-02,
       -3.04823577e-01,  4.62273732e-02,  9.58100855e-01,  1.99162513e-01,
       -6.99615598e-01,  7.50295743e-02,  7.41288900e-01,  3.49739105e-01,
       -3.78860608e-02,  

In [60]:
black_eyed_peas_vector

array([ 0.00980099, -0.00969299,  0.00262159,  0.02069168, -0.00063489,
       -0.00248969,  0.0142802 , -0.00874121,  0.00766608,  0.01062598,
        0.01907556,  0.00112119,  0.01584589, -0.00668949, -0.00683055,
        0.00114164,  0.00097297,  0.00685882, -0.00053361,  0.00967262,
        0.01014591,  0.017445  ,  0.00519766,  0.02537674,  0.01324185,
        0.0164963 ,  0.00390584, -0.00550266, -0.00766674,  0.00293623,
        0.00720947, -0.01457359,  0.00418011,  0.01771929,  0.01165467,
        0.00407373,  0.0135006 ,  0.0277601 ,  0.00377527,  0.00726775,
        0.00408777, -0.01873324,  0.04677689,  0.01126862,  0.01512042,
        0.00201204,  0.01540973,  0.01262689, -0.00394643,  0.01950999,
        0.01005538,  0.0092296 , -0.00092852,  0.02036732,  0.00818191,
        0.01057604,  0.00342858,  0.00350251,  0.01057715, -0.00768928,
       -0.00445824,  0.00921123, -0.01591516,  0.01171241, -0.00524268,
        0.00773662, -0.01544316,  0.00171208, -0.0122699 , -0.00

In [61]:
yesica_vector

array([ 2.8390515e-01,  8.7768269e-01, -1.2283803e+00,  7.2134012e-01,
        7.9538591e-02, -3.9269600e+00,  3.3378017e-01, -3.5677894e-03,
        6.7285192e-01,  3.8012370e-01,  1.8228196e-01,  1.0250983e+00,
       -4.6418700e-01,  3.4198898e-01,  1.9271722e-01, -1.1249981e+00,
       -9.2522699e-01,  4.1758567e-02, -3.3936548e-01,  9.2598319e-01,
       -7.2903961e-01,  2.4212823e+00,  1.4307694e-01, -8.0616367e-01,
        1.0133916e+00,  1.3855484e+00,  7.0932406e-01,  8.3371580e-01,
        1.2022414e+00,  1.0122802e+00, -2.1645600e-01,  3.8399270e-01,
       -1.4236994e-01, -7.7313788e-02, -6.7046607e-01, -9.1132277e-01,
       -4.9961887e-02,  1.6128483e+00, -2.0122585e+00,  4.9815800e-02,
        3.2485023e-01, -4.8077139e-01,  3.7448293e-01, -6.7104042e-01,
        1.0990922e-01, -3.0423430e-01,  2.2631412e+00,  3.1480376e-02,
       -1.2733409e+00,  1.1952270e+00, -2.7637941e-01,  2.0434642e-01,
        4.9819928e-01,  2.8953989e+00,  3.0941638e-01, -6.2448895e-01,
      

In [62]:
the_beatles_vector

array([ 1.12240044e-02,  1.04083046e-02,  3.59051861e-04,  1.40695376e-02,
       -9.65525862e-03, -1.19117815e-02,  5.10957697e-03,  6.46626344e-03,
        2.14649737e-02,  6.47819135e-03,  5.96549921e-03,  2.13129409e-02,
       -5.63608902e-03,  8.63438399e-05,  1.71756442e-03, -2.73681735e-03,
       -4.62731440e-03,  7.42675830e-03,  9.19142552e-03,  3.94748151e-03,
        2.03168392e-03,  2.12109573e-02,  2.72101327e-03,  2.42761220e-03,
        1.37539953e-02,  2.13951413e-02,  2.67016608e-02,  8.67519155e-03,
        1.70163251e-02,  1.29003571e-02,  1.76429115e-02,  8.22618883e-03,
       -1.14356717e-02,  1.28118983e-02,  4.32938896e-03, -2.21673818e-03,
        7.11844245e-04,  2.65706331e-02, -1.51904775e-02, -3.59046180e-03,
        1.01569286e-02,  2.09837593e-03,  7.77992792e-03, -1.04042925e-02,
        6.40815962e-03,  2.00448139e-03,  2.41006948e-02,  2.00080350e-02,
        8.48771166e-03,  1.32290227e-02,  1.78267842e-03,  2.37035919e-02,
        1.51703125e-02,  

In [63]:
# zimin과 black_eyed_peas를 내적하는 코드
np.dot(zimin_vector, black_eyed_peas_vector)

0.49494734

In [64]:
# yesica와 the beatles를 내적하는 코드
np.dot(yesica_vector, the_beatles_vector)

0.73956734

In [65]:
# 이런, 1이 나와야 될 것 같은데 한참 낮은 수치인 0.49정도가 나왔습니다. 
# factors를 늘리거나 iterations를 늘려야할 것 같습니다. 이후 진행되는 내용은 일단 이렇게 학습된 모델을 사용하겠습니다.
# 만약 이 두 수치를 늘려 1에 가깝게 나왔다면 이 모델이 잘 학습되었다고 볼 수 있을까요?
# 정답은 아직 검증되지 않았다입니다. 학습 데이터에 대해서 fitting은 되었지만 아직 보지 못한 데이터에 대한, 
# 예를 들어 제가 queen을 선호할지에 대한 예측을 잘할지 못할지는 검증하지 않았기 때문입니다.

In [66]:
# 모델이 zimin의 queen에 대한 선호도를 어떻게 예측할지 한 번 보겠습니다
queen = artist_to_idx['queen']
queen_vector = als_model.item_factors[queen]
np.dot(zimin_vector, queen_vector)

0.3247396

In [67]:
# 모델이 yesica의 radiohead에 대한 선호도를 어떻게 예측할지 한 번 보겠습니다
radiohead = artist_to_idx['radiohead']
radiohead_vector = als_model.item_factors[radiohead]
np.dot(yesica_vector, radiohead_vector)

0.7175827

In [68]:
# queen에 대한 선호도를 예측한 결과를 어떻게 해석하면 좋을까요? 
# 사용자와 아티스트 벡터 내적이 0.5를 기준으로 그보다 높으면 모델이 선호한다고 판단했다고 말할 수 있을까요?
# 모델을 만드는 사람이 정하기 나름입니다. 객관적인 지표로 만들어서 기준을 정할 수도 있고 도메인 경험을 통해 정할 수도 있습니다.
# 추천 시스템은 다른 머신러닝 task보다 객관적인 평가가 어려운 분야입니다. 
# 사람을 도통 알 수 없기 때문에 객관적인 지표를 정하기 어렵기 때문입니다.

## 비슷한 아티스트 찾기

In [69]:
# 비슷한 아티스트 찾기
# AlternatingLeastSquares 클래스에 구현되어 있는 similar_items 메서드를 통하여 비슷한 아티스트를 찾습니다. 
# 처음으로는 제가 좋아하는 coldplay로 찾아보겠습니다.

In [70]:
favorite_artist = 'coldplay'
artist_id = artist_to_idx[favorite_artist]
similar_artist = als_model.similar_items(artist_id, N=15)
similar_artist

[(62, 0.1194679),
 (277, 0.11761674),
 (28, 0.11671323),
 (5, 0.11635094),
 (217, 0.11607582),
 (490, 0.115673624),
 (247, 0.11531021),
 (473, 0.11499481),
 (418, 0.11431594),
 (694, 0.11346184),
 (268, 0.11266628),
 (1018, 0.11257646),
 (910, 0.11237443),
 (782, 0.11183047),
 (55, 0.11148347)]

In [71]:
# (아티스트의 id, 유사도) Tuple 로 반환하고 있습니다. 아티스트의 id를 다시 아티스트의 이름으로 매핑시켜 주겠습니다.

In [72]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_artist = {v:k for k,v in artist_to_idx.items()}
[idx_to_artist[i[0]] for i in similar_artist]

['coldplay',
 'muse',
 'the killers',
 'red hot chili peppers',
 'radiohead',
 'oasis',
 'the beatles',
 'placebo',
 'u2',
 'foo fighters',
 'pink floyd',
 'the smashing pumpkins',
 'nirvana',
 'the white stripes',
 'arctic monkeys']

In [74]:
# 몇 번 더 반복해서 확인하기 위해 위의 코드를 함수로 만들고 확인해보겠습니다.

In [75]:
def get_similar_artist(artist_name: str):
    artist_id = artist_to_idx[artist_name]
    similar_artist = als_model.similar_items(artist_id)
    similar_artist = [idx_to_artist[i[0]] for i in similar_artist]
    return similar_artist

In [76]:
# 저는 힙합도 좋아하기 때문에 2pac으로도 확인해 보겠습니다.

In [77]:
[(idx_to_artist[i[0]], i[1]) for i in explain[1]]get_similar_artist('2pac')

['2pac',
 'nas',
 'notorious b.i.g.',
 'dr. dre',
 'dmx',
 'the game',
 'snoop dogg',
 'jay-z',
 '50 cent',
 'ludacris']

In [89]:
get_similar_artist('madonna')

['madonna',
 'kylie minogue',
 'nelly furtado',
 'britney spears',
 'rihanna',
 'moby',
 'u2',
 'depeche mode',
 'christina aguilera',
 'michael jackson']

In [78]:
# 우와! 스눕독, 닥터 드레, 노토리우스, 나스, 제이지 같은 전설들이 추천됩니다. 
# 힙합의 경우 마니아들이 많아서 더 잘 추천되는 것으로 생각됩니다.

# 마니아는 데이터상에서 표현될 때 어떤 특징을 가질까요?
# 마니아들은 특정 장르의 아티스트들에게로 선호도가 집중되고, 
# 다른 장르의 아티스트들과는 선호도가 낮게 나타날 것입니다. 
# 이런 마니아들의 존재로 인해 같은 장르의 아티스트들의 벡터들도 더 가까워져서 
# getsimilarartist시 장르별 특성이 두드러지게 될 것입니다.

In [79]:
get_similar_artist('lady gaga')

['lady gaga',
 'britney spears',
 'rihanna',
 'katy perry',
 'beyoncé',
 'the pussycat dolls',
 'christina aguilera',
 'kelly clarkson',
 'justin timberlake',
 'mariah carey']

In [80]:
# lady gaga의 경우 여자 아티스트들이 추천되고 있습니다.

## 유저에게 아티스트 추천하기

In [81]:
# AlternatingLeastSquares 클래스에 구현되어 있는 recommend 메서드를 통하여 제가 좋아할 만한 아티스트를 추천 받습니다. 
# filter_already_liked_items 는 유저가 이미 평가한 아이템은 제외하는 Argument입니다.

In [82]:
user = user_to_idx['zimin']
# recommend에서는 user*item CSR Matrix를 받습니다.
artist_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
artist_recommended

[(274, 0.45372978),
 (350, 0.44295222),
 (369, 0.4388491),
 (627, 0.42752516),
 (550, 0.42411757),
 (564, 0.4213951),
 (1800, 0.4132448),
 (354, 0.40589088),
 (621, 0.4041928),
 (409, 0.40383446),
 (355, 0.3944359),
 (2249, 0.38819122),
 (5555, 0.37834775),
 (391, 0.37184107),
 (24, 0.3688653),
 (28, 0.35700554),
 (247, 0.34960896),
 (277, 0.34886447),
 (418, 0.34736824),
 (5, 0.3470095)]

In [83]:
[idx_to_artist[i[0]] for i in artist_recommended]

['michael jackson',
 'rihanna',
 'justin timberlake',
 'maroon 5',
 'britney spears',
 'kanye west',
 'lady gaga',
 'nelly furtado',
 'alicia keys',
 'amy winehouse',
 'madonna',
 'katy perry',
 'timbaland',
 'christina aguilera',
 'jack johnson',
 'the killers',
 'the beatles',
 'muse',
 'u2',
 'red hot chili peppers']

In [90]:
user = user_to_idx['yesica']
# recommend에서는 user*item CSR Matrix를 받습니다.
artist_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
artist_recommended

[(277, 0.73269033),
 (5, 0.72430336),
 (910, 0.6925461),
 (418, 0.6854942),
 (473, 0.6823571),
 (268, 0.6814171),
 (28, 0.6663153),
 (531, 0.6639572),
 (75, 0.657406),
 (615, 0.6534744),
 (1170, 0.6467838),
 (490, 0.63557684),
 (694, 0.6331762),
 (773, 0.62138474),
 (776, 0.6150451),
 (1500, 0.6136718),
 (1018, 0.607312),
 (352, 0.6027687),
 (782, 0.5923398),
 (503, 0.58972526)]

In [91]:
[idx_to_artist[i[0]] for i in artist_recommended]

['muse',
 'red hot chili peppers',
 'nirvana',
 'u2',
 'placebo',
 'pink floyd',
 'the killers',
 'depeche mode',
 'queen',
 'moby',
 'daft punk',
 'oasis',
 'foo fighters',
 'nine inch nails',
 'the cure',
 'gorillaz',
 'the smashing pumpkins',
 'linkin park',
 'the white stripes',
 'green day']

In [84]:
# rihanna를 추천해주고 있네요. 모델은 왜 rihanna를 추천해줬을까요? 
# AlternatingLeastSquares 클래스에 구현된 explain 메소드를 사용하면 
# 제가 기록을 남긴 데이터 중 이 추천에 기여한 정도를 확인할 수 있습니다.

In [92]:
rihanna = artist_to_idx['rihanna']
explain = als_model.explain(user, csr_data, itemid=rihanna)

In [93]:
# 이 method는 추천한 콘텐츠의 점수에 기여한 다른 콘텐츠와 기여도(합이 콘텐츠의 점수가 됩니다.)를 반환합니다. 
# 어떤 아티스트들이 이 추천에 얼마나 기여하고 있는 걸까요?

In [94]:
[(idx_to_artist[i[0]], i[1]) for i in explain[1]]

[('madonna', 0.24655657162603503),
 ('the prodigy', 0.04410506490804754),
 ('coldplay', 0.03409945720196632),
 ('the beatles', 0.0012746572534428995),
 ('radiohead', -0.005445130020818168)]

In [88]:
# beyoncé와 black eyed peas가 가장 크게 기여 했네요. 
# 둘 다 흑인 아티스트이고 beyoncé의 경우는 같은 성별이라는 점도 크게 작용하지 않았을까 싶습니다.

# 눈치채신 분도 있겠지만 결과가 조금 이상합니다. filter_already_liked_items=True로 했는데 
# 앞에 maroon 5는 제가 이미 평가한 아이템이 나오고 있습니다.이것은 implicit 버전 0.4.2에서 생긴 버그입니다. 

# 우리가 만든 모델은 몇 가지 아쉬운 점이 있습니다.

# 유저, 아티스트에 대한 Meta 정보를 반영하기 쉽지 않습니다. 연령대별로 음악취향이 굉장히 다르지 않을까요?
# 유저가 언제 play 했는지에 반영하기 쉽지 않습니다. 10년 전에 즐겨듣던 아티스트와 지금 즐겨듣는 아티스트를 비교해보세요.